# Sepsis onset time

- Definition: calculate all the patients. If any of them with SOFA score change within the first antibiotics or blood culture time -48 to +24 hours

In [82]:
import pandas as pd
import numpy as np

In [89]:
sepsis_df = pd.read_csv('../data/sepsis3-df.csv')
print(sepsis_df.shape)
print(sepsis_df.columns)

(61532, 48)
Index(['icustay_id', 'hadm_id', 'excluded', 'intime', 'outtime', 'dbsource',
       'suspected_infection_time_poe', 'suspected_infection_time_poe_days',
       'specimen_poe', 'positiveculture_poe', 'antibiotic_time_poe',
       'blood_culture_time', 'blood_culture_positive', 'age', 'gender',
       'is_male', 'ethnicity', 'race_white', 'race_black', 'race_hispanic',
       'race_other', 'metastatic_cancer', 'diabetes', 'height', 'weight',
       'bmi', 'first_service', 'hospital_expire_flag', 'thirtyday_expire_flag',
       'icu_los', 'hosp_los', 'sepsis_angus', 'sepsis_martin',
       'sepsis_explicit', 'septic_shock_explicit', 'severe_sepsis_explicit',
       'sepsis_nqf', 'sepsis_cdc', 'sepsis_cdc_simple', 'elixhauser_hospital',
       'vent', 'sofa', 'lods', 'sirs', 'qsofa', 'qsofa_sysbp_score',
       'qsofa_gcs_score', 'qsofa_resprate_score'],
      dtype='object')


## Map outtime to the table

In [90]:
# construct dictionary that maps the icustay to the relative time
outtime_dict = dict()

for i in range(sepsis_df.shape[0]):
    icustay_id = sepsis_df.iloc[i]['icustay_id']
    intime = sepsis_df.iloc[i]['intime']
    outtime = sepsis_df.iloc[i]['outtime']
    if pd.isna(outtime):
        print('None! But impossible!')
        continue

    outtime_dict[int(icustay_id)] = (intime, outtime)

None! But impossible!
None! But impossible!
None! But impossible!
None! But impossible!
None! But impossible!
None! But impossible!
None! But impossible!
None! But impossible!
None! But impossible!
None! But impossible!


In [91]:
test_list_df = pd.read_csv('../data/my-in-hospital-mortality2/test/listfile.csv')
test_list_df

,icustay,filename,mortality,mortality_hours,intime,deathtime,outtime
0,200009,29904_episode1_timeseries.csv,0,NaN,2189-11-30 10:34:32,NaN,2189-12-02 14:17:37
1,200025,19167_episode1_timeseries.csv,0,NaN,2113-08-25 09:29:12,NaN,2113-08-28 12:42:58
2,200047,26135_episode1_timeseries.csv,0,NaN,2190-01-30 21:47:36,NaN,2190-01-31 15:06:57
3,200050,78716_episode2_timeseries.csv,0,NaN,2149-07-14 17:51:18,NaN,2149-07-15 12:19:36
4,200063,99052_episode1_timeseries.csv,0,NaN,2141-03-09 23:20:49,NaN,2141-04-05 16:33:28
5,200077,2530_episode2_timeseries.csv,0,NaN,2163-04-08 17:36:00,NaN,2163-04-24 14:40:00
6,200098,73499_episode1_timeseries.csv,0,NaN,2136-03-27 11:50:32,NaN,2136-03-31 17:18:39
7,200130,12411_episode2_timeseries.csv,0,NaN,2184-07-09 02:57:00,NaN,2184-07-10 01:41:00
8,200140,27874_episode1_timeseries.csv,0,NaN,2197-01-18 05:39:31,NaN,2197-02-07 15:00:25
9,200161,30173_episode1_timeseries.csv,0,NaN,2142-03-09 17:26:35,NaN,2142-03-10 18:32:52


In [86]:
test_list_df = pd.read_csv('../data/my-in-hospital-mortality2/test/listfile.csv')
outtime_arr = []
for i in range(test_list_df.shape[0]):
    icustay_id = test_list_df.iloc[i]['icustay']
    if icustay_id not in outtime_dict:
        print('%d not in the dict!' % icustay_id)
    
    intime, outtime = outtime_dict[icustay_id]
    assert intime == test_list_df.iloc[i]['intime']
    
    outtime_arr.append(outtime)

test_list_df['outtime'] = outtime_arr
test_list_df.to_csv('../data/my-in-hospital-mortality2/test/listfile.csv', index=None)

In [92]:
train_list_df = pd.read_csv('../data/my-in-hospital-mortality2/train/listfile.csv')
outtime_arr = []
for i in range(train_list_df.shape[0]):
    icustay_id = train_list_df.iloc[i]['icustay']
    if icustay_id not in outtime_dict:
        print('%d not in the dict!' % icustay_id)
    
    intime, outtime = outtime_dict[icustay_id]
    outtime_arr.append(outtime)

train_list_df['outtime'] = outtime_arr
train_list_df.to_csv('../data/my-in-hospital-mortality2/train/listfile.csv', index=None)

## Find sepsis

In [88]:
sepsis_df

,icustay_id,hadm_id,excluded,intime,outtime,dbsource,suspected_infection_time_poe,suspected_infection_time_poe_days,specimen_poe,positiveculture_poe,...,sepsis_cdc_simple,elixhauser_hospital,vent,sofa,lods,sirs,qsofa,qsofa_sysbp_score,qsofa_gcs_score,qsofa_resprate_score
0,200001,152234,1,2181-11-25 19:06:12,2181-11-28 20:59:25,metavision,2181-11-18 11:10:00,7.330694,BLOOD CULTURE,0.0,...,0,3,0,6,7,3,2,1.0,0.0,1.0
1,200003,163557,1,2199-08-02 19:50:04,2199-08-08 17:09:18,carevue,2199-08-02 21:02:00,-0.049954,BLOOD CULTURE,1.0,...,1,0,1,6,3,4,2,1.0,0.0,1.0
2,200006,189514,1,2159-09-03 11:28:14,2159-09-04 19:08:10,carevue,NaN,NaN,NaN,NaN,...,0,-5,1,1,2,2,2,1.0,0.0,1.0
3,200007,129310,1,2109-02-17 10:03:37,2109-02-18 17:03:12,carevue,NaN,NaN,NaN,NaN,...,0,0,0,1,2,2,2,1.0,0.0,1.0
4,200009,129607,1,2189-11-30 10:34:32,2189-12-02 14:17:37,carevue,NaN,NaN,NaN,NaN,...,0,-4,1,3,4,4,2,1.0,0.0,1.0
5,200010,192256,1,2132-08-04 23:03:19,2132-08-05 22:14:11,metavision,NaN,NaN,NaN,NaN,...,0,-2,0,0,1,2,1,0.0,0.0,1.0
6,200011,121562,1,2188-08-06 01:39:24,2188-08-07 16:50:53,metavision,2188-08-05 21:41:00,0.165556,URINE,1.0,...,1,8,1,3,4,1,1,0.0,0.0,1.0
7,200012,177527,1,2153-12-23 05:12:55,2153-12-23 15:55:54,carevue,2153-12-23 00:30:00,0.196470,BLOOD CULTURE,0.0,...,0,0,0,1,1,3,2,1.0,0.0,1.0
8,200014,127229,1,2105-02-16 23:16:48,2105-02-18 16:53:29,carevue,2105-02-17 00:00:00,-0.030000,BLOOD CULTURE,0.0,...,0,0,1,3,5,4,2,1.0,0.0,1.0
9,200016,117458,1,2150-12-02 15:59:20,2150-12-03 14:54:29,metavision,NaN,NaN,NaN,NaN,...,0,0,0,1,3,2,1,1.0,0.0,0.0


In [28]:
# construct dictionary that maps the icustay to the relative time
icu_to_suspected_time = dict()

for i in range(sepsis_df.shape[0]):
    icustay_id = sepsis_df.iloc[i]['icustay_id']
    rel_time = sepsis_df.iloc[i]['suspected_rel_ini_time']
    if pd.isna(rel_time):
        continue

    icu_to_suspected_time[icustay_id] = rel_time

In [33]:
x_df = pd.read_csv('../data/my-in-hospital-mortality/test/%s' % thefile)
x_df.columns

Index(['Hours', 'Albumin', 'Bicarbonate', 'Bilirubin', 'Blood urea nitrogen',
       'Capillary refill rate', 'Chloride', 'Cholesterol', 'Creatinine',
       'Diastolic blood pressure', 'Fraction inspired oxygen',
       'Glascow coma scale total', 'Glucose', 'Heart Rate', 'Height',
       'Mean blood pressure', 'Oxygen saturation',
       'Partial pressure of oxygen', 'Phosphate', 'Platelets', 'Potassium',
       'Red blood cell count', 'Respiratory rate', 'Sodium',
       'Systolic blood pressure', 'Temperature', 'Troponin-T', 'Urine output',
       'Weight', 'White blood cell count', 'pH'],
      dtype='object')

In [45]:
thefile = '2115_episode1_timeseries.csv'
x_df = pd.read_csv('../data/my-in-hospital-mortality2/test/%s' % thefile)

x_df['Respiratory rate'].describe()

count    138.000000
mean      14.717391
std        9.156378
min        0.000000
25%       12.000000
50%       15.500000
75%       16.000000
max      100.000000
Name: Respiratory rate, dtype: float64

In [30]:
thefile = '2115_episode1_timeseries.csv'
starttime = -15
endtime = 24

x_df = pd.read_csv('../data/my-in-hospital-mortality/test/%s' % thefile)
for i in range(x_df.shape[0]):
    if x_df.iloc[i]['Hours'] < starttime or x_df.iloc[i]['Hours'] > endtime:
        continue
    
    # TODO: calculate sofa score. But seems extremely comlicated!

,Hours,Albumin,Bicarbonate,Bilirubin,Blood urea nitrogen,Capillary refill rate,Chloride,Cholesterol,Creatinine,Diastolic blood pressure,...,Red blood cell count,Respiratory rate,Sodium,Systolic blood pressure,Temperature,Troponin-T,Urine output,Weight,White blood cell count,pH
0,1.141111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN,7.46
1,3.691111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,135.0,NaN,NaN,NaN,NaN,NaN,NaN,7.51
2,4.341111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.50
3,5.307778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.30
4,5.824444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.36
5,7.591111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84.0,...,NaN,14.0,NaN,125.0,34.700001,NaN,NaN,NaN,NaN,NaN
6,7.674444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84.0,...,NaN,14.0,NaN,127.0,34.599999,NaN,160.0,NaN,NaN,NaN
7,7.907778,NaN,24.0,NaN,15.0,NaN,113.0,NaN,0.5,NaN,...,3.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN
8,7.924444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.0,...,NaN,14.0,NaN,116.0,34.800000,NaN,NaN,NaN,NaN,NaN
9,8.157778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,141.0,NaN,NaN,NaN,NaN,NaN,NaN,7.39


In [ ]:
sepsis_arr = []
sepsis_time_arr = []

for i in range(test_list_df.shape[0]):
    icustay_id = test_list_df.iloc[i]['icustay']
    if icustay_id not in icu_to_suspected_time:
        sepsis_arr.append(None)
        sepsis_time_arr.append(None)
    
    rel_time = icu_to_suspected_time[icustay_id]
    start_time = rel_time - 48 # 48 hours before
    end_time = rel_time + 24 # 24 hours after
    
    find_first_sofa_time(test_list_df.iloc[i]['filename'], start_time, end_time)

In [25]:
test_list_df = pd.read_csv('../data/my-in-hospital-mortality/test/listfile.csv')
test_list_df

,icustay,filename,mortality,mortality_hours,intime,deathtime
0,200009,29904_episode1_timeseries.csv,0,NaN,2189-11-30 10:34:32,NaN
1,200025,19167_episode1_timeseries.csv,0,NaN,2113-08-25 09:29:12,NaN
2,200047,26135_episode1_timeseries.csv,0,NaN,2190-01-30 21:47:36,NaN
3,200050,78716_episode2_timeseries.csv,0,NaN,2149-07-14 17:51:18,NaN
4,200063,99052_episode1_timeseries.csv,0,NaN,2141-03-09 23:20:49,NaN
5,200077,2530_episode2_timeseries.csv,0,NaN,2163-04-08 17:36:00,NaN
6,200098,73499_episode1_timeseries.csv,0,NaN,2136-03-27 11:50:32,NaN
7,200130,12411_episode2_timeseries.csv,0,NaN,2184-07-09 02:57:00,NaN
8,200140,27874_episode1_timeseries.csv,0,NaN,2197-01-18 05:39:31,NaN
9,200161,30173_episode1_timeseries.csv,0,NaN,2142-03-09 17:26:35,NaN
